# Analyze Sisma 2016 and 2022 and group all data in a single file

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
# non molto utile, sono i comuni fuori da cratere

df0 = pd.read_csv('sisma2016_data/Elenco dei Comuni Fuori Cratere da cui risultano perventue RCR.CSV',sep=';',na_values=['-'])
df0 = df0.drop(['Provincia'],axis='columns')
df0.head(2)

,Comune,RCR Danni Lievi,RCR Danni Gravi,RCR Totali
0,Acquaviva Picena,3.0,2.0,5
1,Agugliano,NaN,1.0,1


In [3]:
df = pd.read_csv('sisma2016_data/Ricostruzione Privata.CSV',sep=';',na_values=['-'])
df = df.drop(['PROVINCIA','DOMANDE / EDIFICI INAGIBILI DL.1','DOMANDE / EDIFICI INAGIBILI DL'],axis='columns')
df

,Comune,DOMANDE DL,DOMANDE DG,TOT,DI CUI ACCOLTE,CANTIERI AVVIATI,CANTIERI CONCLUSI,DL,DG
0,Monteleone di Fermo,11,NaN,11,3.0,3.0,2.0,13.000,51.000
1,Folignano,57,15.0,72,46.0,46.0,27.0,74.000,102.000
2,Castel di Lama,73,50.0,123,83.0,83.0,43.0,95.000,130.000
3,Fiastra,138,83.0,221,131.0,131.0,75.0,180.000,476.000
4,Esanatoglia,80,9.0,89,55.0,55.0,49.0,106.000,141.000
...,...,...,...,...,...,...,...,...,...
133,Norcia,728,196.0,924,449.0,449.0,230.0,1.324,2.236
134,Cerreto di Spoleto,53,7.0,60,20.0,20.0,8.0,100.000,82.000
135,Montefranco,13,1.0,14,4.0,4.0,NaN,25.000,13.000
136,Preci,94,32.0,126,48.0,48.0,20.0,236.000,418.000


In [4]:

df = df.fillna(0)
#df['DOMANDE DL / DL'] = df['DOMANDE DL'] / df['DL']
#df['DOMANDE DG / DG'] = df['DOMANDE DG'] / df['DG']
df.columns = ['Comune','SMALL DAMAGE REQ.', 'GREAT DAMAGE REQ.', 'TOT REQ.','ACCEPTED REQ.',
              'BUILDING STARTED', 'BUILIDING CONCLUDED','SMALL DAMAGE','GREAT DAMAGE']
df['TOT. DAMAGE'] = df['SMALL DAMAGE'] + df['GREAT DAMAGE']
df.head(2)

,Comune,SMALL DAMAGE REQ.,GREAT DAMAGE REQ.,TOT REQ.,ACCEPTED REQ.,BUILDING STARTED,BUILIDING CONCLUDED,SMALL DAMAGE,GREAT DAMAGE,TOT. DAMAGE
0,Monteleone di Fermo,11,0.0,11,3.0,3.0,2.0,13.0,51.0,64.0
1,Folignano,57,15.0,72,46.0,46.0,27.0,74.0,102.0,176.0


In [5]:
df1 = pd.read_csv('sisma2016_data/Sintesi della Ricostruzione nel Territorio.CSV',sep=';',na_values=['-'])
df1 = df1.drop(['PR','Tecnici','Amministrativi','Altro','Totale ','RCR Presentate VS Attese -tasso di copertura- (%) '],axis='columns')
df1 = df1.fillna(0)
#df1['RCR Presentate VS Attese -tasso di copertura- (%) '] = df1['RCR Presentate '] / df1['RCR Attese ']
df1.columns = ['Comune','RCR FORECAST','RCR ACTUAL','OOPP']
df1.head(2)

,Comune,RCR FORECAST,RCR ACTUAL,OOPP
0,Acquasanta Terme,1295,271.0,2
1,Amandola,936,123.0,5


In [6]:
df1 = df1.merge(df)
df1.head(10)

,Comune,RCR FORECAST,RCR ACTUAL,OOPP,SMALL DAMAGE REQ.,GREAT DAMAGE REQ.,TOT REQ.,ACCEPTED REQ.,BUILDING STARTED,BUILIDING CONCLUDED,SMALL DAMAGE,GREAT DAMAGE,TOT. DAMAGE
0,Acquasanta Terme,1295,271.0,2,226,124.0,350,175.0,175.0,98.0,465.0,834.000,1299.000
1,Amandola,936,123.0,5,140,53.0,193,85.0,85.0,32.0,350.0,596.000,946.000
2,Apiro,442,64.0,20,76,37.0,113,32.0,32.0,17.0,141.0,302.000,443.000
3,Appignano del Tronto,179,36.0,7,49,19.0,68,30.0,30.0,14.0,75.0,105.000,180.000
4,Arquata del Tronto,2070,109.0,11,60,100.0,160,85.0,85.0,40.0,294.0,1.776,295.776
5,Ascoli Piceno,1770,452.0,1,545,167.0,712,323.0,323.0,137.0,827.0,946.000,1773.000
6,Belforte del Chienti,230,62.0,8,49,30.0,79,37.0,37.0,23.0,84.0,148.000,232.000
7,Belmonte Piceno,73,8.0,15,14,1.0,15,7.0,7.0,2.0,26.0,48.000,74.000
8,Bolognola,117,29.0,6,22,11.0,33,15.0,15.0,9.0,63.0,55.000,118.000
9,Caldarola,799,77.0,14,84,39.0,123,61.0,61.0,34.0,199.0,607.000,806.000


In [7]:
tot2022 = pd.read_csv('sisma2016_data/tot2022.csv',na_values=['-'])
tot2022 = tot2022[['Comune','N','Eur']]
tot2022.columns = ['Comune','N_Interventi','Eur_Interventi']
tot2022

,Comune,N_Interventi,Eur_Interventi
0,Abbateggio,7,1472180.02
1,Acciano,2,1400000.00
2,Alanno,27,8457704.19
3,Ancarano,24,9233372.38
4,Anversa degli Abruzzi,15,3101280.22
...,...,...,...
489,Trevi,66,13960950.26
490,Umbertide,5,2439037.75
491,Valfabbrica,19,3364161.42
492,Vallo di Nera,69,12817688.64


In [8]:
priv2022 = pd.read_csv('sisma2016_data/priv2022.csv',na_values=['-'])
priv2022 = priv2022[['COMUNE','N_R_PRESENTATE','EUR_R_PRESENTATE','N_R_DA_PRESENTARE','EUR_R_DA_PRESENTARE']]
priv2022.columns=['Comune','N_Interventi_priv','Eur_Interventi_priv','N_Interventi_priv2','Eur_Interventi_priv2']
priv2022 = priv2022[['Comune','N_Interventi_priv','Eur_Interventi_priv']]
priv2022

,Comune,N_Interventi_priv,Eur_Interventi_priv
0,Abbateggio,3.0,318028.02
1,Alanno,15.0,3082653.79
2,Ancarano,8.0,2629988.38
3,Anversa degli Abruzzi,9.0,954907.12
4,Arsita,19.0,2809990.70
...,...,...,...
436,Trevi,39.0,7745279.96
437,Umbertide,NaN,NaN
438,Valfabbrica,9.0,1170571.44
439,Vallo di Nera,15.0,2475804.08


In [9]:
pubb2022 = pd.read_csv('sisma2016_data/pubb2022.csv',na_values=['-'])
pubb2022 = pubb2022[['COMUNE','N_R_PRESENTATE','EUR_R_PRESENTATE','N_R_DA_PRESENTARE','EUR_R_DA_PRESENTARE']]
pubb2022.columns=['Comune','N_Interventi_pubb','Eur_Interventi_pubb','N_Interventi_pubb2','Eur_Interventi_pubb2']
pubb2022 = pubb2022[['Comune','N_Interventi_pubb','Eur_Interventi_pubb']]
pubb2022

,Comune,N_Interventi_pubb,Eur_Interventi_pubb
0,Alanno,2.0,2444348.40
1,Atri,6.0,19775966.06
2,Barete,1.0,200000.00
3,Basciano,4.0,2318411.50
4,Bellante,NaN,NaN
...,...,...,...
334,Umbertide,1.0,NaN
335,Valfabbrica,1.0,295000.00
336,Vallo di Nera,12.0,2606741.25
337,Valtopina,1.0,575000.00


In [10]:
tot2022[tot2022['Comune']=='Fabriano']

,Comune,N_Interventi,Eur_Interventi
277,Fabriano,363,2.440490e+08


In [11]:
pop2016 = pd.read_csv('quake_data/Classificazioni statistiche-e-dimensione-dei-comuni_2016.csv',sep=';')
pop2016 = pop2016.iloc[:,[3,6]]
pop2016.columns = ['Comune','pop_iniz_2016']
addon = pop2016[pop2016['Comune'].isin(['Fabriano','Cerreto d\'Esi'])]
addon

,Comune,pop_iniz_2016
4898,Cerreto d'Esi,3967
4902,Fabriano,31020


In [12]:
istat = pd.read_csv('quake_data/alldata_reshape.csv')
istat = istat[['Comune','pop_iniz_2016']]
istat = pd.concat([istat,addon])

In [13]:
istat[istat['Comune']=='Fabriano']

,Comune,pop_iniz_2016
4902,Fabriano,31020.0


In [14]:
sisma = df1.merge(tot2022).merge(priv2022).merge(pubb2022).merge(pop2016)
sisma[sisma['Comune']=='Fabriano']

,Comune,RCR FORECAST,RCR ACTUAL,OOPP,SMALL DAMAGE REQ.,GREAT DAMAGE REQ.,TOT REQ.,ACCEPTED REQ.,BUILDING STARTED,BUILIDING CONCLUDED,SMALL DAMAGE,GREAT DAMAGE,TOT. DAMAGE,N_Interventi,Eur_Interventi,N_Interventi_priv,Eur_Interventi_priv,N_Interventi_pubb,Eur_Interventi_pubb,pop_iniz_2016
26,Fabriano,387,98.0,11,134,27.0,161,48.0,48.0,22.0,211.0,177.0,388.0,363,2.440490e+08,185.0,94990149.12,11.0,22448264.7,31020


In [15]:
sisma['DELTA FORECAST ACTUAL'] =   sisma['RCR FORECAST'] - sisma['RCR ACTUAL']
sisma['DELTA G DAMAGE Interventi'] =   sisma['GREAT DAMAGE REQ.'] - sisma['N_Interventi']
sisma['P RCR'] = sisma['RCR ACTUAL'] / sisma['RCR FORECAST']

In [16]:
to_norm = ['RCR FORECAST','RCR ACTUAL','DELTA FORECAST ACTUAL','DELTA G DAMAGE Interventi','OOPP','SMALL DAMAGE REQ.',
           'GREAT DAMAGE REQ.','TOT REQ.','ACCEPTED REQ.',
           'BUILDING STARTED','BUILIDING CONCLUDED',
           'SMALL DAMAGE','GREAT DAMAGE','TOT. DAMAGE',
           'N_Interventi','Eur_Interventi',
           'N_Interventi_priv','Eur_Interventi_priv',
           'N_Interventi_pubb','Eur_Interventi_pubb',
          ]
for n in to_norm:
    sisma[n] = sisma[n] / sisma['pop_iniz_2016']
sisma
sisma.to_csv('quake_data/sisma_all.csv',index=False)